# Laboratorio 09

Integrantes:
- Ricardo Méndez 21289
- Sara Echeverría 21371
- Francisco Castillo 21562
- Melissa Pérez 21385

Repositorio: https://github.com/MelissaPerez09/Laboratorio09-CC3045

## Task 01 - Teoría

### 1. Diga cual es la diferencia entre Modelos de Markov y Hidden Markov Models

La diferencia es que los Modelos de Markov son modelos estocásticos que describen la secuencia de un evento probabilístico futuro que depende solo del evento actual. Mientras que los HMM son una extensión de los Modelos de Markov que incluyen estados ocultos, donde a pesar de no poder observarlos directamente se puede inferir de los datos "visibles". Entonces, la principal diferencia es que los HMM permiten modelas sistemas donde no todos los aspectos dle proceso son directamente observables. [(Jurafsky, 2024)](https://web.stanford.edu/~jurafsky/slp3/A.pdf)

### 2. Investigue qué son los factorial HMM (Hidden Markov Models)

Son un tipo de modelo probabilístico utilizado para aprender modelos de series temporal. Son una generalización de los HMMs, en los que la variable oculta se factoriza en múltiples variables de estado y, por lo tanto, se representa de manera distribuida. En un FHMM, la variable oculta es representada por un conjunto de variables de estado, cada una de las cuales está asociada con una serie de probabilidades de transición, que dan la probabilidad de que cada componente se active dado el componente activo anterior. La salida del modelo en cada paso de tiempo depende de los valores de las variables de estado de todas las cadenas. 

Los FHMMs son instancias de modelos generativos de múltiples causas, ya que la salida del modelo en cada paso de tiempo depende de los valores de las variables de estado de todas las cadenas. Son también un caso especial de redes bayesianas dinámicas (DBNs). [(Ghahrmani, 1997)](https://www.ee.columbia.edu/~sfchang/course/svia-F03/papers/factorial-HMM-97.pdf)

### 3. Especifique en sus propias palabras el algoritmo Forward Backward para HMM

### 4. En el algoritmo de Forward Backward, por qué es necesario el paso de Backward (puede escribir ejemplos o casos para responder esta pregunta)

## Task 02 - Algoritmo Forward Backward en HMM

En este ejercicio estamos ante un modelo meteorológico representado por un Modelo Oculto de Markov (HMM) con dos estados: "Soleado" y "Lluvioso". Queremos predecir el tiempo en un día determinado basándonos en las observaciones de si el día anterior estuvo soleado o lluvioso.

In [1]:
import numpy as np

class HMM:
    def __init__(self, states, observations, initial_prob, transition_prob, emission_prob):
        self.states = states
        self.observations = observations
        self.initial_prob = initial_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob

    def generate_sequence(self, length):
        sequence = []
        current_state = np.random.choice(self.states, p=[self.initial_prob[state] for state in self.states])

        for _ in range(length):
            sequence.append(current_state)
            current_state = np.random.choice(self.states, p=[self.transition_prob[current_state][next_state] for next_state in self.states])

        return sequence

    def forward(self, observations):
        alpha = np.zeros((len(observations), len(self.states)))

        # Initialize base cases (t = 0)
        for i, state in enumerate(self.states):
            alpha[0, i] = self.initial_prob[state] * self.emission_prob[state][observations[0]]

        # Run forward algorithm for t > 0
        for t in range(1, len(observations)):
            for j, state in enumerate(self.states):
                alpha[t, j] = sum(alpha[t-1, i] * self.transition_prob[self.states[i]][state] for i in range(len(self.states))) * self.emission_prob[state][observations[t]]

        return alpha

    def backward(self, observations):
        beta = np.zeros((len(observations), len(self.states)))

        # Initialize base cases (T)
        beta[len(observations) - 1] = np.ones(len(self.states))

        # Run backward algorithm
        for t in range(len(observations) - 2, -1, -1):
            for i in range(len(self.states)):
                beta[t, i] = sum(beta[t+1, j] * self.transition_prob[self.states[i]][self.states[j]] * self.emission_prob[self.states[j]][observations[t+1]] for j in range(len(self.states)))

        return beta

    def compute_state_probabilities(self, observations):
        alpha = self.forward(observations)
        beta = self.backward(observations)
        prob = (alpha * beta) / np.sum(alpha * beta, axis=1, keepdims=True)
        return prob

# Definición de parámetros
states = ['Sunny', 'Rainy']
observations = ['Sunny', 'Sunny', 'Rainy']
initial_prob = {'Sunny': 0.5, 'Rainy': 0.5}
transition_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.4, 'Rainy': 0.6}}
emission_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.3, 'Rainy': 0.7}}

# Creación de la instancia de HMM
hmm = HMM(states, observations, initial_prob, transition_prob, emission_prob)

# Generar una secuencia de observaciones
obs_sequence = hmm.generate_sequence(5)
print("Secuencia Generada:", obs_sequence)

# Cálculo de probabilidades forward
forward_probs = hmm.forward(observations)
print("\nProbabilidades Forward:")
print(forward_probs)

# Cálculo de probabilidades backward
backward_probs = hmm.backward(observations)
print("\nProbabilidades Backward:")
print(backward_probs)

# Calcular probabilidades de estado
state_probs = hmm.compute_state_probabilities(observations)
print("\nProbabilidades de Estados:")
print(state_probs)

Secuencia Generada: ['Rainy', 'Sunny', 'Sunny', 'Sunny', 'Sunny']

Probabilidades Forward:
[[0.4     0.15   ]
 [0.304   0.051  ]
 [0.05272 0.06398]]

Probabilidades Backward:
[[0.222 0.186]
 [0.3   0.5  ]
 [1.    1.   ]]

Probabilidades de Estados:
[[0.76092545 0.23907455]
 [0.781491   0.218509  ]
 [0.45175664 0.54824336]]
